In [4]:
import pandas as pd
import re

%matplotlib inline

In [5]:
with open('db_con.txt') as a:
    db_con = a.read()

## Get all the proposals and their themes 

In [3]:
proposals = pd.read_sql("""
select proposals.*,
    recipients.name as recipient_name,
    recipients.description as recipient_description,
    recipients.category_code as recipient_category_code,
    recipients.operating_for as recipient_operating_for,
    recipients.income_band as recipient_income_band
from proposals
    inner join recipients
        on proposals.recipient_id = recipients.id
""", con=db_con, index_col='id')

In [45]:
proposal_themes = pd.read_sql("""
select proposal_id,
    theme_id,
    themes.name,
    parent_themes.name as parent_name
from proposal_themes 
    inner join themes 
        on proposal_themes.theme_id = themes.id
    left outer join themes as parent_themes
        on parent_themes.id = themes.parent_id
""", con=db_con)
proposal_themes.loc[:, "parent_name"] = proposal_themes['parent_name'].fillna(proposal_themes['name'])
proposal_themes

,proposal_id,theme_id,name,parent_name
0,1,16,Education and training,Education and training
1,1,37,Public and societal benefit,Public and societal benefit
2,2,31,Disability,Health and medicine
3,2,39,Disaster preparedness and relief,Public and societal benefit
4,2,40,Food and agriculture,Public and societal benefit
...,...,...,...,...
115524,40773,49,Family and relationships,Social welfare
115525,40773,52,Isolation and loneliness,Social welfare
115526,40773,33,"Mental wellbeing, diseases and disorders",Health and medicine
115527,40773,21,Older people,Groups


In [49]:
themes = proposal_themes[["proposal_id", "parent_name"]].drop_duplicates()
themes

,proposal_id,parent_name
0,1,Education and training
1,1,Public and societal benefit
2,2,Health and medicine
3,2,Public and societal benefit
5,2,Social welfare
...,...,...
115521,40772,Employment
115522,40772,Education and training
115524,40773,Social welfare
115526,40773,Health and medicine


In [41]:
proposal_districts = pd.read_sql("""
select proposal_id,
    district_id,
    districts.name as "district_name",
    districts.region as "region_name",
    districts.sub_country as "uk_country_name",
    countries.name as "country_name"
from districts_proposals 
    inner join districts 
        on districts_proposals.district_id = districts.id
    inner join countries
        on districts.country_id = countries.id
""", con=db_con)
proposal_districts.loc[:, "geoname"] = proposal_districts['region_name'].fillna(
    proposal_districts['uk_country_name']
).fillna("Overseas")
proposal_districts

,proposal_id,district_id,district_name,region_name,uk_country_name,country_name,geoname
0,1,11766,Cheltenham,South West,England,United Kingdom,South West
1,1,11843,Gloucester,South West,England,United Kingdom,South West
2,1,11864,"Herefordshire, County of",West Midlands,England,United Kingdom,West Midlands
3,1,12049,Tewkesbury,South West,England,United Kingdom,South West
4,1,12094,Worcester,West Midlands,England,United Kingdom,West Midlands
...,...,...,...,...,...,...,...
1732092,40770,12029,Stockton-on-Tees,North East,England,United Kingdom,North East
1732093,40770,11967,Redcar and Cleveland,North East,England,United Kingdom,North East
1732094,40771,15356,Wales,None,None,United Kingdom,Overseas
1732095,40772,11876,Isle of Anglesey,None,Wales,United Kingdom,Wales


In [43]:
region = proposal_districts[["proposal_id", "geoname"]].drop_duplicates()
region

,proposal_id,geoname
0,1,South West
2,1,West Midlands
5,2,Overseas
6,3,Overseas
7,4,East Midlands
...,...,...
1732091,40769,South East
1732092,40770,North East
1732094,40771,Overseas
1732095,40772,Wales


In [69]:
s = region.groupby('proposal_id').size()
s[s > 9].index

Int64Index([   26,   102,   222,   296,   395,   400,   405,   420,   426,
              444,
            ...
            37148, 37149, 37164, 37165, 37227, 37232, 37279, 37304, 37331,
            39928],
           dtype='int64', name='proposal_id', length=1164)

In [102]:
england = [
    "East Midlands",
    "East of England",
    "London",
    "North East",
    "North West",
    "South East",
    "South West",
    "West Midlands",
    "Yorkshire and The Humber"
]
uk = england + [
    "Scotland",
    "Northern Ireland",
    "Wales",
]

s = pd.crosstab(
    region['proposal_id'],
    region['geoname']
)
area = pd.Series(index=s.index)
area = area.fillna(s[s.sum(axis=1)==1].idxmax(axis=1))
area[(s[uk].sum(axis=1)==len(uk)) & area.isnull()] = "UK wide"
area[(s[uk + ["Overseas"]].sum(axis=1)==len(uk)+1) & area.isnull()] = "UK and overseas"
area[(s['Overseas']==1) & (s.sum(axis=1)>1) & area.isnull()] = "UK and overseas"
area[(s[england].sum(axis=1)==len(england)) & area.isnull()] = "England"
area[area.isnull()] = "England"

area.value_counts(dropna=False)

England                     4390
Overseas                    2737
London                      1800
South East                  1138
North West                  1127
Scotland                    1072
South West                   995
UK wide                      913
West Midlands                790
East of England              735
Yorkshire and The Humber     694
East Midlands                670
Wales                        587
UK and overseas              578
North East                   391
Northern Ireland             186
dtype: int64

In [107]:
proposals = proposals.join(area.rename("area"))

## Add in proper values for proposals

In [123]:
CATEGORIES = {
#     'Grant funding': {
      201: 'Capital',
      202: 'Revenue - Core',
      203: 'Revenue - Project',
#     },
#     'Other': {
      101: 'Other',
#     }
}
categories_index = pd.CategoricalDtype(CATEGORIES.values(), ordered=True)


RECIPIENT_CATEGORIES = {
    # 'Unincorporated': {
        201: 'A community or voluntary group',
        202: 'An unincorporated association',
        203: 'An unregistered charity',
    # },
    # 'Incorporated': {
        301: 'A charitable organisation',
        302: 'A company',
        303: 'An industrial and provident society (IPS)',
    # },
    # 'Other': {
        101: 'An individual',
        102: 'Another type of organisation',
    # }
}
recipient_categories_index = pd.CategoricalDtype(RECIPIENT_CATEGORIES.values(), ordered=True)

INCOME_BANDS = {
    0: 'Less than £10k', # { label: 'Less than £10k', min: 0,        max: 9999 },
    1: '£10k - £99k', # { label: '£10k - £99k',    min: 10000,    max: 99999 },
    2: '£100k - £999k', # { label: '£100k - £999k',  min: 100000,   max: 999999 },
    3: '£1m - £10m', # { label: '£1m - £10m',     min: 1000000,  max: 10000000 },
    4: 'More than £10m', # { label: 'More than £10m', min: 10000001, max: float('inf') }
}
income_bands_index = pd.CategoricalDtype(INCOME_BANDS.values(), ordered=True)

OPERATING_FOR = {
    0: 'Yet to start',
    1: 'Less than 12 months',
    2: 'Less than 3 years',
    3: '4 years or more'
}
operating_for_index = pd.CategoricalDtype(OPERATING_FOR.values(), ordered=True)

In [124]:
AMOUNT_BINS = [0, 500, 1000, 2000, 5000, 10000, 100000, 1000000, float("inf")]
AMOUNT_BIN_LABELS = ["Under £500", "£500 - £1k", "£1k - £2k", "£2k - £5k", "£5k - £10k",
                    "£10k - £100k", "£100k - £1m", "Over £1m"]

In [125]:
DURATION_BINS = [0, 11, 15, 23, 35, 46, float("inf")]
DURATION_BIN_LABELS = ["Under 12 months", "12 months", "18 months", "2 years", "3 years", "4 years +"]

In [126]:
proposals.loc[:, "category"] = proposals.category_code.apply(CATEGORIES.get).astype(categories_index)
proposals.loc[:, "recipient_category"] = proposals.recipient_category_code.apply(RECIPIENT_CATEGORIES.get).astype(recipient_categories_index)
proposals.loc[:, "recipient_operating_for"] = proposals.recipient_operating_for.apply(OPERATING_FOR.get).astype(operating_for_index)
proposals.loc[:, "recipient_income_band"] = proposals.recipient_income_band.apply(INCOME_BANDS.get).astype(income_bands_index)

In [128]:
proposals.loc[:, "amount_bins"] = pd.cut(proposals['min_amount'], bins=AMOUNT_BINS, labels=AMOUNT_BIN_LABELS)
proposals.loc[:, "duration_bins"] = pd.cut(proposals['min_duration'], bins=DURATION_BINS, labels=DURATION_BIN_LABELS)

## Save as pickles

In [129]:
proposals.to_pickle('proposals.pkl')
themes.to_pickle('themes.pkl')